# DATA WRANGLING

Note: It is important to say that this is not the first data wrangling attemp. Previous to this notoebook I did a data analysis where I discovered some patterns and some special cases and where I made sure of aspects like all transfers csv had the same format before merge them. That is the reason why you will find this notebook clean and clear othewrwise It would be a mess ;) 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import re

In [2]:
stats = pd.read_csv("../data/players_stats.csv") # player stats

tl = pd.read_csv("../data/primera-division.csv") # Transfer la liga
tp = pd.read_csv("../data/premier-league.csv") # transfers premier
ts = pd.read_csv("../data/serie-a.csv") # transfers serie a
tb = pd.read_csv("../data/bundesliga.csv") # transfers bundesliga
tl1 = pd.read_csv("../data/ligue-1.csv") # transfers ligue 1

In [3]:
print("Transfers la liga shape:" , tl.shape)
print("Transfers premier shape:" , tp.shape)
print("Transfers serie a shape:" , ts.shape)
print("Transfers bundeliga shape:" , tb.shape)
print("Transfers bundeliga shape:" , tl1.shape)

Transfers la liga shape: (17255, 12)
Transfers premier shape: (23498, 12)
Transfers serie a shape: (28211, 12)
Transfers bundeliga shape: (14032, 12)
Transfers bundeliga shape: (17387, 12)


In [4]:
transf = pd.concat([tl,tp,ts,tb,tl1]) # merge both leagues transfers

In [5]:
transf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100383 entries, 0 to 17386
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   club_name           100383 non-null  object 
 1   player_name         100383 non-null  object 
 2   age                 100344 non-null  float64
 3   position            100383 non-null  object 
 4   club_involved_name  100383 non-null  object 
 5   fee                 100345 non-null  object 
 6   transfer_movement   100383 non-null  object 
 7   transfer_period     100383 non-null  object 
 8   fee_cleaned         40381 non-null   float64
 9   league_name         100383 non-null  object 
 10  year                100383 non-null  int64  
 11  season              100383 non-null  object 
dtypes: float64(2), int64(1), object(9)
memory usage: 10.0+ MB


In [6]:
stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35656 entries, 0 to 35655
Data columns (total 36 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      35656 non-null  int64  
 1   name            35656 non-null  object 
 2   club            35656 non-null  object 
 3   age             35656 non-null  int64  
 4   position        35656 non-null  object 
 5   apps            35656 non-null  object 
 6   mins            35656 non-null  int64  
 7   goals           35656 non-null  object 
 8   assists         35656 non-null  object 
 9   yel             35656 non-null  object 
 10  red             35656 non-null  object 
 11  shots           35656 non-null  object 
 12  ps%             35656 non-null  object 
 13  aerials_won     35656 non-null  object 
 14  motm            35656 non-null  object 
 15  rating          35656 non-null  float64
 16  tackles         35628 non-null  object 
 17  interceptions   35628 non-null 

### Data preparation

Previous to start with the transformations we have to execute some filters in trasnfers dataframe:
 * Get only the seasons where we have player stats. Note: The transfers made in the 2010/2011 season were based in 2009/2010 season stats. That is whu we will get seasons + 1, from 2010/2011 to 2022/2023, despite having players stats from 2009/2010 to 2021/2022.
 
 
 * Get only summer and outgoing trades because we have players stats for full season, not mid seasons. Winter transfers may have more correlation with mid seasons stats.
    
 * Remove end of loan tranfers because this trades may have nothing to do with player stats.

In [7]:
# Get seasons
seasons = ['2010/2011','2011/2012','2012/2013','2013/2014','2014/2015','2015/2016','2016/2017','2017/2018','2018/2019','2019/2020','2020/2021','2021/2022','2022/2023']
transf = transf[transf['season'].isin(seasons)]

In [8]:
# Get summer and out
transf = transf[transf['transfer_movement'].str.upper() == 'OUT']
transf = transf[transf['transfer_period'].str.upper() == 'SUMMER']

In [9]:
# Remove end of loans
transf = transf[~transf['fee'].str.contains('End of loan')]

### Transformations and Feature Engineering

The first step is make club names from both dataframes equal because in the transfers we have the full name in the league´s language and in the stats dataframe the short name in english.

In [10]:
## Clubs with different names in both df
clubs_transf = transf['club_name'].unique() 
clubs_stats = stats['club'].unique()

print(np.setdiff1d(clubs_stats,clubs_transf))
print(len(np.setdiff1d(clubs_stats,clubs_transf)))

['AC Carpi' 'Almeria' 'Amiens' 'Angers' 'Arles-Avignon' 'Arsenal'
 'Atalanta' 'Atletico' 'Augsburg' 'Auxerre' 'Barcelona' 'Bari' 'Bayern'
 'Benevento' 'Birmingham' 'Blackburn' 'Blackpool' 'Bochum' 'Bologna'
 'Bolton' 'Bordeaux' 'Borussia M.Gladbach' 'Boulogne' 'Bournemouth'
 'Brentford' 'Brescia' 'Brest' 'Brighton' 'Burnley' 'Cadiz' 'Caen'
 'Cagliari' 'Cardiff' 'Catania' 'Celta Vigo' 'Cesena' 'Chelsea' 'Chievo'
 'Clermont Foot' 'Cordoba' 'Crotone' 'Darmstadt' 'Deportivo'
 'Deportivo Alaves' 'Dijon' 'Eibar' 'Elche' 'Empoli' 'Espanyol' 'Everton'
 'Evian' 'FC Koln' 'Fiorentina' 'Fortuna Duesseldorf' 'Freiburg'
 'Frosinone' 'Fulham' 'Genoa' 'Getafe' 'Girona' 'Granada' 'Grenoble'
 'Greuther Fuerth' 'Guingamp' 'Hamburg' 'Hannover' 'Hercules'
 'Hertha Berlin' 'Hoffenheim' 'Huddersfield' 'Hull' 'Ingolstadt' 'Inter'
 'Juventus' 'Kaiserslautern' 'Las Palmas' 'Lazio' 'Le Mans' 'Lecce'
 'Leeds' 'Leganes' 'Leicester' 'Lens' 'Levante' 'Leverkusen' 'Lille'
 'Liverpool' 'Livorno' 'Lorient' 'Lyon' 'Mai

In [11]:
# Remove some characters that are usually present in a football club
transf['clubs_short'] = transf['club_name'].apply(lambda x: x.strip().replace('CF','').replace('UD','')
                                                             .replace('CD','').replace('RCD','').replace('CA','')
                                                              .replace('SD','').replace('FC','').replace('á','a')
                                                              .replace('é','e').replace('í','i').replace('ó','o')
                                                              .replace('ú','u').replace('City','').replace('AC','')
                                                              .replace('United','Utd').replace('Town','')
                                                              .replace('SS','').replace('BC','').replace('AS','')
                                                              .replace('Calcio','').replace('US','').replace('BSC','')
                                                              .replace('VfB','').replace('SV','').replace('FSV','')
                                                              .replace('ST','').replace('RB','').replace('SC','')
                                                              .replace('SpvGG','').replace('VfL','').replace('96','')
                                                              .replace('98','').replace('04','').replace('RC','')
                                                              .replace('LO','').replace('OGC','').replace('AJ','')
                                                              .replace('29','').replace('SM','').replace('EA','')
                                                              .replace('Stade','').strip()
                                                             ) 

In [12]:
clubs = {
    "Atletico de Madrid":"Atletico",
    "Blackburn Rovers":"Blackburn",
    "Bolton Wanderers":"Bolton",
    "A Bournemouth":"Bournemouth",
    "Brighton & Hove Albion":"Brighton",
    "Celta de Vigo":"Celta Vigo",
    "Deportivo de La Coruña":"Deportivo",
    "R Espanyol Barcelona":"Espanyol",
    "Leeds Utd":"Leeds",
    "R Mallorca":"Mallorca",
    "Manchester":"Man City",
    "Manchester Utd":"Man Utd",
    "Newcastle Utd":"Newcastle",
    "Queens Park Rangers":"QPR",
    "Real Betis Balompie":"Real Betis",
    "Huesca":"SD Huesca",
    "Sheffield Utd":"Sheff Utd",
    "Sunderland A":"Sunderland",
    "Tottenham Hotspur":"Tottenham",
    "West Bromwich Albion":"West Brom",
    "West Ham Utd":"West Ham",
    "Wigan Athletic":"Wigan",
    "Wolverhampton Wanderers":"Wolves",
    "Milan":"AC Milan",
    "Carpi  1909":"AC Carpi",
    "Bologna  1909":"Bologna",
    "Chievo Verona":"Chievo",
    "A Fiorentina":"Fiorentina",
    "Catania S":"Catania",
    "Genoa C":"Genoa",
    "Inter Milan":"Inter",
    "Hellas Verona":"Verona",
    "C Napoli":"Napoli",
    "Novara  1908":"Novara",
    "Delfino Pescara 1936":"Pescara",
    "Salernitana 1919":"Salernitana",
    "UC Sampdoria":"Sampdoria",
    "Parma  1913":"Parma Calcio 1913",
    "Bayern Munich":"Bayern",
    "Borussia Mönchengladbach":"Borussia M.Gladbach",
    "Darmstadt 98":"Darmstadt",
    "1.  Köln":"FC Koln",
    "Fortuna Düsseldorf":"Fortuna Duesseldorf",
    "SpVgg Greuther Fürth":"Greuther Fuerth",
    "Hamburger":"Hamburg",
    "Hertha":"Hertha Berlin",
    "TSG 1899 Hoffenheim":"Hoffenheim",
    "1. Kaiserslautern":"Kaiserslautern",
    "Bayer  Leverkusen":"Leverkusen",
    "1.F Mainz 05":"Mainz",
    "1. Nuremberg":"Nuernberg",
    "Paderborn 07":"Paderborn",
    "Leipzig":"RBL",
    "1. Union Berlin":"Union Berlin",
    "G Ajaccio":"GFC Ajaccio",
    "Angers O":"Angers",
    "Athletic Club Arlesien":"Arles-Avignon",
    "Girondins Bordeaux":"Bordeaux",
    "Clermont Foot 63":"Clermont Foot",
    "Dijon O":"Dijon",
    "Évian Thonon Gaillard":"Evian",
    "Grenoble Foot 38":"Grenoble",
    "Le Mans Union Club 72":"Le Mans",
    "Olympique Lyon":"Lyon",
    "Olympique Marseille":"Marseille",
    "Montpellier H":"Montpellier",
    "Nancy-Lorraine":"Nancy",
    "Paris Saint-Germain":"PSG",
    "Brestois":"Brest",
    "Rennais":"Rennes",
    "Bastia":"SC Bastia",
    "Saint-Étienne":"Saint-Etienne",
    "Sochaux-Montbeliard":"Sochaux",
    "Strasbourg Alsace":"Strasbourg",
    "E Troyes":"Troyes",
    "Nîmes Olympique":"Nimes",
    "Ajaccio":"AC Ajaccio"

}

In [13]:
transf = transf.replace({'clubs_short':clubs} )

In [14]:
clubs_transf = transf['clubs_short'].unique() 
clubs_stats = stats['club'].unique()

print(np.setdiff1d(clubs_stats,clubs_transf))
print(len(np.setdiff1d(clubs_stats,clubs_transf)))

['Boulogne' 'Grenoble' 'Le Mans' 'Portsmouth' 'Tenerife' 'Xerez']
6


This 6 teams are present in stats dataframe but not in tranfer dataframe because they played in of these leagues in the season 2009/2010 (the first one we have) but they were relegated and never came back to the top league of their country.

The next step is to equal both dataframe name leagues.

In [15]:
print("Transfers")
print(transf['league_name'].unique())
print("Stats")
stats['league'].unique()

Transfers
['Primera Division' 'Premier League' 'Serie A' '1 Bundesliga' 'Ligue 1']
Stats


array(['Bundesliga', 'LaLiga', 'Ligue 1', 'Premier League', 'Serie A'],
      dtype=object)

In [16]:
transf['league_name'] = transf['league_name'].replace('Primera Division','LaLiga').replace('1 Bunseliga','Bundesliga')

For player names we will apply an enconding to remove accents and special characters.

In [17]:
stats['name']=stats['name'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
transf['player_name']=transf['player_name'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

As we mentioned before, we want to join the transfers seasons + 1 with the stats seasons. For that we will create a column with the season minus 1.

In [18]:
transf['season_less1'] = transf['season'].apply(lambda x: str(int(x.split('/')[0]) - 1) + '/' + str(int(x.split('/')[1]) - 1))

### Join

In [19]:
join = pd.merge(transf,stats,how='inner',left_on=['clubs_short','player_name','season_less1'],right_on=['club','name','season'])
join

,club_name,player_name,age_x,position_x,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,...,fouled,offsides,dispossed,bad_controls,avg_passes,crosses,long_passes,through_passes,league,season_y
0,RCD Espanyol Barcelona,Nicolas Pareja,26.0,Centre-Back,Spartak Moscow,€10.00m,out,Summer,10.0,LaLiga,...,0.7,-,0.3,-,43.5,0.1,4.4,0.1,LaLiga,2009/2010
1,RCD Espanyol Barcelona,Moises Hurtado,29.0,Defensive Midfield,Olympiacos,€1.80m,out,Summer,1.8,LaLiga,...,0.9,-,0.7,-,39.2,0.1,2.9,-,LaLiga,2009/2010
2,RCD Espanyol Barcelona,Fernando Marques,25.0,Right Winger,Parma,Loan fee:€200Th.,out,Summer,0.2,LaLiga,...,1.3,0.2,1,-,13.6,0.8,0.6,0.1,LaLiga,2009/2010
3,RCD Espanyol Barcelona,Raul Tamudo,32.0,Centre-Forward,Real Sociedad,free transfer,out,Summer,0.0,LaLiga,...,0.5,1,0.7,-,10.5,-,0.2,-,LaLiga,2009/2010
4,RCD Espanyol Barcelona,Ben Sahar,20.0,Centre-Forward,Hapoel Tel Aviv,loan transfer,out,Summer,NaN,LaLiga,...,0.8,0.7,0.5,-,6.5,0.1,0.2,-,LaLiga,2009/2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7141,Clermont Foot 63,Jason Berthomier,32.0,Attacking Midfield,Valenciennes FC,free transfer,out,summer,0.0,Ligue 1,...,0.5,0.1,0.6,1.3,44.4,1.3,1.7,0.1,Ligue 1,2021/2022
7142,Clermont Foot 63,Arthur Desmas,28.0,Goalkeeper,AC Le Havre,?,out,summer,NaN,Ligue 1,...,0.3,-,-,-,24.1,-,2.7,-,Ligue 1,2021/2022
7143,Clermont Foot 63,Yadaly Diaby,22.0,Attacking Midfield,A. Lustenau,loan transfer,out,summer,NaN,Ligue 1,...,0.2,-,0.2,0.4,3.6,-,-,-,Ligue 1,2021/2022
7144,Clermont Foot 63,Jerome Phojo,29.0,Right-Back,Without Club,-,out,summer,NaN,Ligue 1,...,-,-,-,-,3,-,-,-,Ligue 1,2021/2022


There is one special case that we have to deal with. The tranfers datasets have the trades between two clubs: the "owner" of the player and the one that is adquiring him. The special case is that there are some occasions in which the player did not play the season with his club but was loaned out the season prior to transferring him.

In [20]:
# Get the players that did not match
outer = pd.merge(transf,stats,how='outer',left_on=['clubs_short','player_name','season_less1'],right_on=['club','name','season'],indicator=True)
outer = outer.query('_merge != "both"')
outer = outer[outer['club_name'].notnull()]

In [21]:
# Create a dataframe with the name of the club where the player played the specific season
played_for = stats[['name','club','season']]
played_for.columns = ['name_pf','club_pf','season_pf']
played_for

,name_pf,club_pf,season_pf
0,Arjen Robben,Bayern,2009/2010
1,Stefan Kieling,Leverkusen,2009/2010
2,Zvjezdan Misimovic,Wolfsburg,2009/2010
3,Edin Dzeko,Wolfsburg,2009/2010
4,Claudio Pizarro,Werder Bremen,2009/2010
...,...,...,...
35651,Luca Vignali,Spezia,2021/2022
35652,Vincenzo Fiorillo,Salernitana,2021/2022
35653,Ramzi Aya,Salernitana,2021/2022
35654,Gianluca Pegolo,Sassuolo,2021/2022


In [22]:
# Join with stats using the club they played for 
outers = pd.merge(outer,played_for,how='left',left_on=['player_name','season_less1'],right_on=['name_pf','season_pf'])
join2 = pd.merge(outers[['player_name','club_name','club_pf','club_involved_name','fee','season_less1']],stats,how='inner',left_on=['club_pf','player_name','season_less1'],right_on=['club','name','season'])
join2

,player_name,club_name,club_pf,club_involved_name,fee,season_less1,Unnamed: 0,name,club,age,...,fouled,offsides,dispossed,bad_controls,avg_passes,crosses,long_passes,through_passes,league,season
0,Valdo,RCD Espanyol Barcelona,Malaga,Levante,free transfer,2009/2010,337,Valdo,Malaga,29,...,0.8,0.1,1.3,-,18.3,0.3,0.5,0.1,LaLiga,2009/2010
1,Milan Smiljanic,RCD Espanyol Barcelona,Sporting Gijon,Partizan,loan transfer,2009/2010,446,Milan Smiljanic,Sporting Gijon,24,...,0.7,-,0.2,-,23.5,0.3,1.2,0.2,LaLiga,2009/2010
2,Roman Martinez,RCD Espanyol Barcelona,Tenerife,Tigre,loan transfer,2009/2010,106,Roman Martinez,Tenerife,27,...,1.7,0.1,1.8,-,32.7,0.1,2.4,0.1,LaLiga,2009/2010
3,Pedro,Sporting Gijón,Barcelona,Albacete,free transfer,2009/2010,61,Pedro,Barcelona,23,...,1.7,0.6,1.4,-,28.3,0.5,0.8,0.2,LaLiga,2009/2010
4,Renan,Valencia CF,Xerez,Internacional,Loan fee:€300Th.,2009/2010,221,Renan,Xerez,25,...,0.1,-,-,-,28,-,9.5,-,LaLiga,2009/2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,Pietro Pellegri,AS Monaco,Torino,Torino,€5.00m,2021/2022,567,Pietro Pellegri,Torino,21,...,0.7,0.4,0.9,2.2,4.7,-,0.1,-,Serie A,2021/2022
1456,Jean-Eudes Aholou,AS Monaco,Strasbourg,R. Strasbourg,€3.00m,2021/2022,376,Jean-Eudes Aholou,Strasbourg,28,...,0.8,-,0.4,0.8,16.2,0.1,0.3,-,Ligue 1,2021/2022
1457,Benjamin Lecomte,AS Monaco,Atletico,Espanyol,loan transfer,2021/2022,584,Benjamin Lecomte,Atletico,31,...,-,-,-,-,-,-,-,-,LaLiga,2021/2022
1458,Mamadou Camara,RC Lens,Troyes,SC Bastia,loan transfer,2021/2022,519,Mamadou Camara,Troyes,21,...,-,-,0.3,0.3,4.3,-,0.3,-,Ligue 1,2021/2022


We added 1461 players to the inital join.

Note: The reason why I didn´t do this before the first is because to obtain the team they played for I can only use the features season and player name. That makes that if 2 players have the same name and played the same season will be duplicated so I decided to do it just with the ones that didn´t match to minimize the risk.

In [23]:
# Create club_short to have the same name in both datasets
join2['clubs_short'] = join2['club_pf']

In [24]:
# Concat both dataframes
transfers = pd.concat([join,join2],ignore_index=True)
transfers

,club_name,player_name,age_x,position_x,club_involved_name,fee,transfer_movement,transfer_period,fee_cleaned,league_name,...,avg_passes,crosses,long_passes,through_passes,league,season_y,club_pf,age,position,season
0,RCD Espanyol Barcelona,Nicolas Pareja,26.0,Centre-Back,Spartak Moscow,€10.00m,out,Summer,10.0,LaLiga,...,43.5,0.1,4.4,0.1,LaLiga,2009/2010,NaN,NaN,NaN,NaN
1,RCD Espanyol Barcelona,Moises Hurtado,29.0,Defensive Midfield,Olympiacos,€1.80m,out,Summer,1.8,LaLiga,...,39.2,0.1,2.9,-,LaLiga,2009/2010,NaN,NaN,NaN,NaN
2,RCD Espanyol Barcelona,Fernando Marques,25.0,Right Winger,Parma,Loan fee:€200Th.,out,Summer,0.2,LaLiga,...,13.6,0.8,0.6,0.1,LaLiga,2009/2010,NaN,NaN,NaN,NaN
3,RCD Espanyol Barcelona,Raul Tamudo,32.0,Centre-Forward,Real Sociedad,free transfer,out,Summer,0.0,LaLiga,...,10.5,-,0.2,-,LaLiga,2009/2010,NaN,NaN,NaN,NaN
4,RCD Espanyol Barcelona,Ben Sahar,20.0,Centre-Forward,Hapoel Tel Aviv,loan transfer,out,Summer,NaN,LaLiga,...,6.5,0.1,0.2,-,LaLiga,2009/2010,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8601,AS Monaco,Pietro Pellegri,NaN,NaN,Torino,€5.00m,NaN,NaN,NaN,NaN,...,4.7,-,0.1,-,Serie A,NaN,Torino,21.0,FW,2021/2022
8602,AS Monaco,Jean-Eudes Aholou,NaN,NaN,R. Strasbourg,€3.00m,NaN,NaN,NaN,NaN,...,16.2,0.1,0.3,-,Ligue 1,NaN,Strasbourg,28.0,DMC,2021/2022
8603,AS Monaco,Benjamin Lecomte,NaN,NaN,Espanyol,loan transfer,NaN,NaN,NaN,NaN,...,-,-,-,-,LaLiga,NaN,Atletico,31.0,GK,2021/2022
8604,RC Lens,Mamadou Camara,NaN,NaN,SC Bastia,loan transfer,NaN,NaN,NaN,NaN,...,4.3,-,0.3,-,Ligue 1,NaN,Troyes,21.0,Forward,2021/2022


### Final Dataset

Finally, we have to join this players with the stats dataset because we just want the stats dataset plus 2 columns indicating wether tha player was transfered or not and and another one with the fee transfer.

In [25]:
final= pd.merge(stats,
                transfers[['player_name','clubs_short','fee','season_less1']],
                how='left',
                left_on=['name','club','season'],
                right_on=['player_name','clubs_short','season_less1'])

In [26]:
#remove unecesary columns from transfers
final = final.drop(['player_name','clubs_short','season_less1'],axis=1)

In [27]:
final['traded'] = np.where(final['fee'].isna(),0,1)

### Addition: Club position

This part is an addition because doing the EDA I realized that club name can be tricky as a feature so I decided to scrap the league classification and replace the club name by the club league position on this league and hopefully will be more useful.

In [28]:
lc = pd.read_csv("../data/leagues_classification.csv") # leagues classification

In [29]:
lc

,Unnamed: 0,position,club,league,season
0,0,1,BayernMunich,Bundesliga,2009/2010
1,1,2,Schalke,Bundesliga,2009/2010
2,2,3,WerderBremen,Bundesliga,2009/2010
3,3,4,BayerLeverkusen,Bundesliga,2009/2010
4,4,5,BorussiaDortmund,Bundesliga,2009/2010
...,...,...,...,...,...
2561,15,16,Sampdoria,Serie A,2021/2022
2562,16,17,Salernitana,Serie A,2021/2022
2563,17,18,Cagliari,Serie A,2021/2022
2564,18,19,Genoa,Serie A,2021/2022


The dataset has all the teams without spaces between their names so Bayern Munich is BayernMunich. Removing this space in the final dataframe just for the join would be really helpful.

In [30]:
final['club_join'] = final['club'].apply(lambda x: x.replace(' ','').strip() )

In [31]:
poss_clubs = lc.club.unique()
final_clubs = final.club_join.unique()
print(np.setdiff1d(poss_clubs,final_clubs))
print(len(np.setdiff1d(poss_clubs,final_clubs)))

['ArlesAvignon' 'AtleticoMadrid' 'BayerLeverkusen' 'BayernMunich'
 'BorussiaMGladbach' 'DeportivoLaCoruna' 'HamburgerSV' 'ManchesterCity'
 'ManchesterUnited' 'ParisSaintGermain' 'ParmaCalcio' 'QueensParkRangers'
 'RBLeipzig' 'SPAL' 'SaintEtienne' 'SheffieldUnited' 'StPauli'
 'ThononEvianGrandGeneve' 'VfBStuttgart' 'WestBromwich' 'Wolverhampton']
21


In [32]:
final['club_join'] = final['club_join'].apply(lambda x: x.replace('Leverkusen','BayerLeverkusen')
                                              .replace('Bayern','BayernMunich')
                                              .replace('Deportivo','DeportivoLaCoruna')
                                              .replace('Hamburg','HamburgerSV')
                                              .replace('Manchester','Man')
                                              .replace('United','Utd')
                                              .replace('PSG','ParisSaintGermain')
                                              .replace('QPR','QueensParkRangers')
                                              .replace('RBL','RBLeipzig')
                                              .replace('SPAL2013','SPAL')
                                              .replace('ParmaCalcio1913','ParmaCalcio')
                                              .replace('Saint-Etienne','SaintEtienne')
                                              .replace('St.Pauli','StPauli')
                                              .replace('Stuttgart','VfBStuttgart')
                                              .replace('WestBrom','WestBromwich')
                                              .replace('Wolves','Wolverhampton')
                                              .replace('Atletico','AtleticoMadrid')
                                              .replace('Arles-Avignon','ArlesAvignon')
                                              .replace('BorussiaM.Gladbach','BorussiaMGladbach')
                                              .replace('ManCity','ManchesterCity')
                                              .replace('ManUtd','ManchesterUnited')
                                              .replace('SheffUtd','SheffieldUnited')
                                              .replace('Evian','ThononEvianGrandGeneve')
                                              .replace('DeportivoLaCorunaAlaves','DeportivoAlaves')
                                              .strip() )

In [33]:
final_clubs = final.club_join.unique()
print(np.setdiff1d(poss_clubs,final_clubs))
print(len(np.setdiff1d(poss_clubs,final_clubs)))

[]
0


Now both cloumns match at 100% we can make the merge !

In [34]:
lc.columns = lc.columns.str.replace('position','club_position')

In [35]:
final= pd.merge(final,
                lc[['club_position','club','season']],
                how='left',
                left_on=['club_join','season'],
                right_on=['club','season'])

In [36]:
final = final.drop(['Unnamed: 0','club_join','club_y'],axis=1)
final.columns = final.columns.str.replace('club_x','club')

In [37]:
# Create a csv
final.to_csv('../data/players_and_transf.csv',index=False)

Finally we obtained the final dataset which now has the club position in his league in the specific season